# <img src="https://img.icons8.com/dusk/64/000000/artificial-intelligence.png" style="height:50px;display:inline"> EE 046202 - Technion - Unsupervised Learning & Data Analysis
---

#### <a href="https://lioritan.github.io">Lior Friedman</a>

## Tutorial 10 - Contrastive Learning Continues

### <img src="https://img.icons8.com/bubbles/50/000000/checklist.png" style="height:50px;display:inline"> Agenda
---
* [Bootstrap Your Own Latent (BYOL)](#-Bootstrap-Your-Own-Latent-(BYOL))
* [Barlow Twins](#-Barlow-Twins)
* [Lightly - implementing contrastive learning](#-Lightly---implementing-contrastive-learning)
* [Recommended Videos](#-Recommended-Videos)
* [Credits](#-Credits)

In [2]:
# imports for the tutorial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib notebook

### <img src="https://img.icons8.com/dusk/64/000000/paper.png" style="height:50px;display:inline">  Reminder: Contrastive learning
---
* Similar things should be close, giving low loss, and dissimilar things should be far.
* Use augmentations to find positive samples (similar things).
* <img src="./assets/selfsup_contrast_augs.png" style="height:200px;">
* Contrastive loss: collect a batch of $N$ samples and use $N-1$ as negative samples each time.
* <img src="./assets/selfsup_infonce_loss.png" style="height:100px;">

### <img src="https://img.icons8.com/dusk/64/000000/mountain.png" style="height:50px;display:inline"> Bootstrap Your Own Latent (BYOL)
---
* <a href="https://arxiv.org/abs/2006.07733">Bootstrap Your Own Latent (BYOL)</a> creates two views of the data and tries to predict one from the other.
* Unlike previous contrastive methods **there are no negative samples**.
* **TODO!!!!!**

### <img src="https://img.icons8.com/bubbles/50/000000/yin-yang.png" style="height:50px;display:inline"> Barlow Twins
---
* **TODO!!!** (Deep Canonical Correlation)

### <img src="./assets/selfsup_lightly.png" style="height:50px;display:inline"> Lightly - implementing contrastive learning
---
* Lightly SSL is a computer vision framework for self-supervised learning. 
* Contains Pytorch-based implementations for many popular models, including everything we talked about.
* **TODO**

### <img src="https://img.icons8.com/bubbles/50/000000/video-playlist.png" style="height:50px;display:inline"> Recommended Videos
---
#### <img src="https://img.icons8.com/cute-clipart/64/000000/warning-shield.png" style="height:30px;display:inline"> Warning!
* These videos do not replace the lectures and tutorials.
* Please use these to get a better understanding of the material, and not as an alternative to the written material.

#### Video By Subject

* 

## <img src="https://img.icons8.com/dusk/64/000000/prize.png" style="height:50px;display:inline"> Credits
---
* <a href="https://github.com/taldatech/ee046211-deep-learning/blob/main/ee046211_tutorial_09_self_supervised_representation_learning.ipynb"> ee045211 - Deep Learning </a> @ Technion
* <a href="https://lilianweng.github.io/posts/2021-05-31-contrastive/"> Weng, Lilian. (May 2021). Contrastive representation learning. Lil’Log </a>
* A Cookbook of Self-Supervised Learning, Balestriero et al. 2023
* <a href="https://paperswithcode.com/method/byol">Bootstrap Your Own Latent (BYOL)</a>
* <a href="https://paperswithcode.com/method/barlow-twins">Barlow Twins</a>